# before executing this notebook, please download and prepare the Libri-Adapt data using following script file

```bash libri-adapt-download.sh```

In [1]:
import os
import config
import progressbar
import requests
import config
import json
import pandas as pd
from datasets import load_metric

In [5]:
data = "libri-adapt"
domain = "matrix" # "matrix", "nexus6", "pseye", "respeaker", "shure", "usb"
result_file = "results/" + data + "_" + domain + ".csv"

test_df= pd.read_csv(os.path.join(config.root_dir, config.domains[domain]))

In [6]:
test_files = list(test_df["wav_filename"])
test_files = [test_file.replace("/data/libriadapt/en-us/clean", config.root_dir) for test_file in test_files]
gold_text = list(test_df["transcript"])

In [ ]:
encoder_text = list()
decoder_text = list()
encoder_text_lexicon = list()
i = 0
new_gold_text = list()
for test_file in progressbar.progressbar(test_files):
    with open(test_file, "rb") as audio_bytes:
        files = {"audio_file": audio_bytes}
        enhancer_url = config.enhancer_end_point
        req = requests.post(enhancer_url, files=files)
        enhanced_audio = req.content
        transcriber_url = config.transcriber_end_point
        files = {"audio_file": enhanced_audio}
        req = requests.post(transcriber_url, params={'lexicon': "False", "text": ""}, files=files)
        if req.text != 'Internal Server Error':
            result = json.loads(req.text)
            encoder_text.append(result['encoder_text'])
            decoder_text.append(result['decoder_text'])
            encoder_text_lexicon.append(result['decoder_text_lexicon'])
            new_gold_text.append(gold_text[i])
        else:
            pass

gold_text = new_gold_text

In [ ]:
results = {"truth": gold_text,
           "encoder_text": encoder_text,
           "decoder_text": decoder_text,
           "decoder_lexicon": encoder_text_lexicon}
data_frame = pd.DataFrame.from_dict(results)
data_frame.to_csv(result_file, sep="\t")

In [ ]:
df = pd.read_csv(result_file, sep="\t")
metric = load_metric("wer")

In [ ]:
references = list(df["truth"])
references = [reference.lower() for reference in references]

predictions = list(df["encoder_text"])

metric.compute(predictions=predictions, references=references)

In [ ]:
references = list(df["truth"])
references = [reference.lower() for reference in references]

predictions = list(df["decoder_text"])

metric.compute(predictions=predictions, references=references)

In [ ]:
references = list(df["truth"])
references = [reference.lower() for reference in references]

predictions = list(df["decoder_lexicon"])

metric.compute(predictions=predictions, references=references)